In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Embedding, Input, Reshape, Dense, Lambda, Dropout, Flatten, MaxPooling2D,MaxPooling1D, Conv2D, \
    BatchNormalization, LSTM, GRU, Embedding, Conv1D,SimpleRNN, TimeDistributed, SimpleRNN, ConvLSTM2D, CuDNNLSTM, \
    CuDNNGRU, Bidirectional, Activation, MaxPool2D, AveragePooling2D, RepeatVector, AtrousConvolution1D, MaxPooling1D, Activation
from mimo_channels_data_generator2 import RandomChannelMimoDataGenerator
from keras.callbacks import TensorBoard
#from matplotlib import pyplot as plt
import argparse
from keras.models import model_from_json
from keras.constraints import max_norm
import numpy.linalg as la
from keras import backend as K
import os
import shutil
import sys
import json
from datetime import datetime
from scipy.io import savemat, loadmat

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)

# training parameters
epochs = 10

# Parameters
global Nt
global Nr
Nr = 8  # num of Rx antennas, will be larger than Nt for uplink massive MIMO
Nt = 8  # num of Tx antennas
# the sample is a measurement of Y values, and their collection composes an example. The channel estimation
min_randomized_snr_db = -1
max_randomized_snr_db = 1

# must be done per example, each one having a matrix of Nr x numSamplesPerExample of complex numbers
numSamplesPerExample = 256 # number of channel uses, input and output pairs
numExamplesWithFixedChannel = 1 # if wants a gradient calculated many times with same channel
numSamplesPerFixedChannel = numExamplesWithFixedChannel * numSamplesPerExample  # coherence time
batch_size = 5 # obs: it may make sense to have the batch size equals the coherence time
##### consider to use train_test_split from sklearn
num_test_examples = 500  # for evaluating in the end, after training
num_validation_examples =  200 # get small number to avoid slowing down the simulation, test in the end
num_training_examples = 1000

method = 'manual'
file = 'channel_data60Ghz_mob.mat'

In [3]:
training_generator = RandomChannelMimoDataGenerator(batch_size=batch_size, Nr=Nr, Nt=Nt,
                                                    #num_clusters=num_clusters,
                                                    numSamplesPerFixedChannel=numSamplesPerFixedChannel,
                                                    #numSamplesPerExample=numSamplesPerExample, SNRdB=SNRdB,
                                                    numSamplesPerExample=numSamplesPerExample,
                                                    # method='random')
                                                    method=method,
                                                    file = file)
training_generator.randomize_SNR = True
training_generator.min_randomized_snr_db = min_randomized_snr_db
training_generator.max_randomized_snr_db = max_randomized_snr_db

_, dataset = training_generator.get_examples(num_training_examples)

global H_normalization_factor
H_normalization_factor = np.sqrt(Nr * Nt)

In [6]:
# normalize
from sklearn.preprocessing import MinMaxScaler
dataset = dataset.reshape(-1,16*8)
scaler = MinMaxScaler()
scaler.fit(dataset)
dataset = scaler.transform(dataset)
print(np.max(dataset), np.min(dataset))


1.0 0.0


In [ ]:
# transform to time serie


In [ ]:
# split train and test